In [1]:
!pip install riotwatcher

In [76]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import random
import time
import requests.exceptions


In [3]:
 TIERS = ["IRON", "SILVER", "GOLD", "PLATINUM", "DIAMOND"]
RANKS = ["I", "II", "III", "IV"]
REGIONS = ["eun1"]
PAGES = [1]

In [34]:
class Karma:
  def __init__(self, API_KEY):
    self.API_KEY = API_KEY
    self.lol_watcher = LolWatcher(API_KEY)
  
  def player_stats(self, region, summoner):
        """
    {'name': 'Spikal',
    'summonerLevel': 491,
    'tierRank': 'EMERALD IV',
    'wins': 98,
    'losses': 90,
    'veteran': False,
    'inactive': False,
    'freshBlood': False,
    'hotStreak': False}
        """
    player = self.lol_watcher.summoner.by_name(region, summoner)
    ranked_stats = self.lol_watcher.league.by_summoner(region, player['id'])
    columns = [
        "name",
        "summonerLevel",
        "tierRank",
        "wins",
        "losses",
        "veteran",
        "inactive",
        "freshBlood",
        "hotStreak"
    ]

    data = {}
    # player = lol_watcher.summoner.by_name('eun1', 'spikal')
    data['name'] = player['name']
    data['summonerLevel'] = player['summonerLevel']
    if len(ranked_stats) > 0:
      for i in range(0,len(ranked_stats)):
        if 'RANKED_SOLO_5x5' in ranked_stats[i]['queueType']:
          if 'tier' in ranked_stats[i] and 'rank' in ranked_stats[i]:
            t = ranked_stats[i]['tier'] + ' ' + ranked_stats[i]['rank']
            data['tierRank'] = t
          else:
            data['tierRank'] = None

          if 'wins' in ranked_stats[i]:
            t = ranked_stats[i]['wins']
            data['wins'] = t
          else:
            data['wins'] = None

          if 'losses' in ranked_stats[i]:
            t = ranked_stats[i]['losses']
            data['losses'] = t
          else:
            data['losses'] = None

          if 'veteran' in ranked_stats[i]:
            t = ranked_stats[i]['veteran']
            data['veteran'] = t
          else:
            data['veteran'] = None

          if 'inactive' in ranked_stats[i]:
            t = ranked_stats[i]['inactive']
            data['inactive'] = t
          else:
            data['inactive'] = None

          if 'freshBlood' in ranked_stats[i]:
            t = ranked_stats[i]['freshBlood']
            data['freshBlood'] = t
          else:
            data['freshBlood'] = None

          if 'hotStreak' in ranked_stats[i]:
            t = ranked_stats[i]['hotStreak']
            data['hotStreak'] = t
          else:
            data['hotStreak'] = None
    else:
      data['tierRank'] = None
      data['wins'] = None
      data['losses'] = None
      data['veteran'] = None
      data['inactive'] = None
      data['freshBlood'] = None
      data['hotStreak'] = None
    return data

  def playerStatsLastGames(self, region, summoner, start_index, end_index):
        """
        {'average_kills': 2.3,
         'average_deaths': 9.166666666666666,
         'average_assists': 14.533333333333333,
         'average_gold_earned': 8809.133333333333}
        """
        player = self.lol_watcher.summoner.by_name(region, summoner)
        player_puuid = player['puuid']

        #Fetch recent matchList with Pagination
        matchlist=[]
        matchlist = self.lol_watcher.match.matchlist_by_puuid(region, player_puuid, start_index, end_index)

        # Define the initial values for stats
        total_kills = 0
        total_deaths = 0
        total_assists = 0
        total_gold_earned = 0
        total_matches = 0

        for match in matchlist:
            match_details = self.lol_watcher.match.by_id(region, match)
            for participant in match_details['info']['participants']:
                if participant['puuid'] == player_puuid:
                    total_kills += participant['kills']
                    total_deaths += participant['deaths']
                    total_assists += participant['assists']
                    total_gold_earned += participant['goldEarned']
                    total_matches += 1

        #Calculate the average stats
        average_kills = total_kills / total_matches
        average_deaths = total_deaths / total_matches
        average_assists = total_assists / total_matches
        average_gold_earned = total_gold_earned / total_matches
        data = {
            'average_kills':average_kills,
            'average_deaths': average_deaths,
            'average_assists': average_assists,
            'average_gold_earned': average_gold_earned
            
        }
        return data
    

In [41]:
api_key= "RGAPI-c7cf2424-731f-4a9a-8f1f-f6985e62dac3"
region = 'eun1'
summoner = 'spikal'

karma = Karma(api_key)
summonerStats = karma.player_stats(region, summoner)

In [38]:
statsLastGames = karma.playerStatsLastGames(region, summoner, 0, 30)

In [99]:
data = dict(summonerStats)
data.update(statsLastGames)
statsLastGames

{'average_kills': 2.3,
 'average_deaths': 9.166666666666666,
 'average_assists': 14.533333333333333,
 'average_gold_earned': 8809.133333333333}

In [84]:
def get_random_players(api_key, NUM_PLAYERS_PER_RANK):
    # Retrieve random players for each division
    REGIONS = ["na1", "euw1", "eun1", "kr", "br1", "la1", "la2", "oce", "tr1", "ru"]
    TIERS = ["IRON", "BRONZE", "SILVER", "GOLD", "PLATINUM", "DIAMOND"]
    DIVISIONS = ["I", "II", "III", "IV"]

    lol_watcher = LolWatcher(api_key)

    # Initialize an empty list to store players
    all_players = []

    # Define rate limit values
    REQUESTS_PER_10_SECONDS = 500
    REQUESTS_PER_10_MINUTES = 30000

    # Calculate sleep duration based on rate limits
    sleep_duration_10_seconds = 10 / REQUESTS_PER_10_SECONDS
    sleep_duration_10_minutes = 600 / REQUESTS_PER_10_MINUTES

    # Loop through regions, tiers, and divisions
    for region in REGIONS:
        for tier in TIERS:
            for division in DIVISIONS:
                retry_count = 0
                while retry_count < 3:  # Retry up to 3 times in case of errors
                    try:
                        # Retrieve player IDs for the current region, tier, and division
                        league_entries = lol_watcher.league.entries(region, "RANKED_SOLO_5x5", tier, division)

                        # Shuffle the list of players to get random players
                        random.shuffle(league_entries)

                        # Take the first 100 players (or fewer if there are fewer players available)
                        selected_players = league_entries[:NUM_PLAYERS_PER_RANK]

                        # Extract player names and regions and add them to the overall list
                        players_info = [{"name": entry["summonerName"], "region": region} for entry in selected_players]
                        all_players.extend(players_info)

                        # Sleep for a short time to avoid rate limiting
                        time.sleep(sleep_duration_10_seconds)  # Sleep for 10 seconds

                        break  # Success, exit the retry loop
                    except requests.exceptions.HTTPError as e:
                        if e.response.status_code == 429:  # Rate limit exceeded
                            # Wait for a short time and retry
                            time.sleep(sleep_duration_10_seconds)  # Sleep for 10 seconds
                            retry_count += 1
                        elif e.response.status_code == 443:  # Retry for SSL handshake errors
                            retry_count += 1
                        else:
                            print(f"Error: {e.response.status_code}")
                            break  # Break the retry loop for other errors
                    except requests.exceptions.ConnectionError as ce:
                        # Connection error, wait and retry
                        time.sleep(sleep_duration_10_seconds)  # Sleep for 10 seconds
                        retry_count += 1

    # Now, all_players contains a list of dictionaries with player names and regions
    return all_players

In [81]:
#TODO: Problem with 443 with API
train_data = get_random_players(api_key, NUM_PLAYERS_PER_RANK = 200)

In [85]:
test_data = get_random_players(api_key, NUM_PLAYERS_PER_RANK = 50)

In [86]:
len(test_data)

10800

In [ ]:
# Define the column names
# columns = ["name", "region", "summonerLevel", "tierRank", "wins", "losses", "veteran", "inactive", "freshBlood", "hotStreak"
#           , "average_kills", "average_deaths", "average_assists", "average_gold_earned"]

# Create an empty dictionary with columns as keys and empty lists as values'

train_data_stats = []
for x in test_data:
    temp = {}
    summoner = x['name']
    region = x['region']
#     {'name': 'Spikal',
#     'summonerLevel': 491,
#     'tierRank': 'EMERALD IV',
#     'wins': 98,
#     'losses': 90,
#     'veteran': False,
#     'inactive': False,
#     'freshBlood': False,
#     'hotStreak': False}
    summonerStats = karma.player_stats(region, summoner)
    
#     {'average_kills': 2.3,
#          'average_deaths': 9.166666666666666,
#          'average_assists': 14.533333333333333,
#          'average_gold_earned': 8809.133333333333}
    statsLastGames = karma.playerStatsLastGames(region, summoner, 0, 30)
    temp = {'name': summonerStats['name'],
        'region': region,
        'summonerLevel': summonerStats['summonerLevel'],
        'tierRank': summonerStats['tierRank'],
         'wins': summonerStats['wins'],
         'losses': summonerStats['losses'],
         'veteran': summonerStats['veteran'],
         'inactive': summonerStats['inactive'],
         'freshBlood': summonerStats['freshBlood'],
         'hotStreak': summonerStats['hotStreak'],
         'average_kills': statsLastGames['average_kills'],
         'average_deaths': statsLastGames['average_deaths'],
         'average_assists': statsLastGames['average_assists'],
         'average_gold_earned': statsLastGames['average_gold_earned']
         }
    train_data_stats.append(temp)

In [105]:
train_data_stats

[{'name': 'Eos Dawn',
  'region': 'na1',
  'summonerLevel': 292,
  'tierRank': 'IRON I',
  'wins': 3,
  'losses': 2,
  'veteran': False,
  'inactive': False,
  'freshBlood': False,
  'hotStreak': False,
  'average_kills': 7.666666666666667,
  'average_deaths': 7.3,
  'average_assists': 17.466666666666665,
  'average_gold_earned': 12519.833333333334},
 {'name': 'oh yas daddy XD',
  'region': 'na1',
  'summonerLevel': 123,
  'tierRank': 'IRON I',
  'wins': 15,
  'losses': 15,
  'veteran': False,
  'inactive': False,
  'freshBlood': False,
  'hotStreak': False,
  'average_kills': 7.366666666666666,
  'average_deaths': 5.9,
  'average_assists': 7.3,
  'average_gold_earned': 12352.866666666667}]